## Get Candlestick Data 

In [1]:
import requests
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
now = datetime.now()

In [3]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [4]:
def find_candle_trend (candle_no):
    if candle_no < 10:
        return 0
    
    data = pd.read_csv(filename)
    data = data.iloc[candle_no-5:candle_no+1]
#    display (data[['Open','Close','Low','High']])
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = int(r2_score(Y, y_pred)*100)
#    print(Current_Market_Fit)
    coeficient = (linear_regressor.coef_)
    
    if coeficient > 0:
        Current_Market = 1 *  Current_Market_Fit## Bullish / Buy ##
    else:
        Current_Market = -1 * Current_Market_Fit ## Bearish / Sell ##
    
    return Current_Market

In [5]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    instrument = cfg['currency']['instrument']

In [6]:
print('Instrument:', instrument)

Instrument: USD_CHF


### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### Select the Currency Pair

In [8]:
Load_10K_Records=True
Load_20K_Records=False

#  'EUR_USD','USD_CAD','EUR_GBP','EUR_AUD','EUR_CHF',
#  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD',
#  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','EUR_CAD',
#  'USD_CHF','CAD_CHF','NZD_CAD','AUD_USD','EUR_NZD',
#  'NZD_CHF',

currency_pairs = [instrument]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/{}/orders'.format(account_number)
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

# Get Candlesticks Data

In [10]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [11]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [12]:
if Load_10K_Records:
    
    #### Load 10K data
        
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
    datetime_object_15K = parser.parse(second_response['candles'][0]['time'])
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    
    if Load_20K_Records:

        #### Load 15K data

        date= datetime_object_15K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        third_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(third_response['candles'][0]['time'])
        third_response= third_response['candles']
        third_response.extend(second_response)

        #### Load 20K data

        date= datetime_object_20K - relativedelta(years=3)  
        from_date = date.replace(tzinfo=timezone.utc).timestamp()
        params_date = (
            ('count', '5000'),
            ('price', price_char),
            ('from', from_date),
            ('granularity', timeframe),)

        fourth_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                       headers=headers,
                                       params=params_date).json()
        datetime_object_20K = parser.parse(fourth_response['candles'][0]['time'])
        fourth_response= fourth_response['candles']
        fourth_response.extend(third_response)

        response=fourth_response
    else:
        response=second_response 
else:
    response=first_response['candles']

In [13]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),(result[4]),(result[5]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date','Time','f_time','julian_date','Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [14]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [15]:
data.shape

(9665, 12)

In [16]:
data.head(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-12-27,22:00:00,22,361,6,Sunday,2015-12-27T22:00:00.000000000Z,901,0.98661,0.98772,0.98579,0.98646
1,2015-12-28,02:00:00,2,362,0,Monday,2015-12-28T02:00:00.000000000Z,460,0.98646,0.98684,0.98583,0.98598
2,2015-12-28,06:00:00,6,362,0,Monday,2015-12-28T06:00:00.000000000Z,1233,0.98596,0.98906,0.98596,0.98846
3,2015-12-28,10:00:00,10,362,0,Monday,2015-12-28T10:00:00.000000000Z,1446,0.98852,0.98914,0.98686,0.98846
4,2015-12-28,14:00:00,14,362,0,Monday,2015-12-28T14:00:00.000000000Z,1708,0.98845,0.99024,0.98730,0.98814


In [17]:
data.tail(5)

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9660,2022-03-11,02:00:00,2,70,4,Friday,2022-03-11T02:00:00.000000000Z,6495,0.93049,0.93096,0.92951,0.93068
9661,2022-03-11,06:00:00,6,70,4,Friday,2022-03-11T06:00:00.000000000Z,14714,0.93068,0.93209,0.92950,0.93184
9662,2022-03-11,10:00:00,10,70,4,Friday,2022-03-11T10:00:00.000000000Z,21746,0.93188,0.93204,0.92956,0.93144
9663,2022-03-11,14:00:00,14,70,4,Friday,2022-03-11T14:00:00.000000000Z,18826,0.93144,0.93473,0.93124,0.93408
9664,2022-03-11,18:00:00,18,70,4,Friday,2022-03-11T18:00:00.000000000Z,9554,0.93406,0.93500,0.93340,0.93484


## Simple Moving Average (SMA)

In [18]:
data['SMA_5'] = data['Close'].rolling(window=5).mean().round(4)
data['SMA_10'] = data['Close'].rolling(window=10).mean().round(4)
data['SMA_20'] = data['Close'].rolling(window=20).mean().round(4)

## Simple Moving Average Range

In [19]:
data['F_SMA_5'] = data['Close'] - data['SMA_5']
data['F_SMA_10'] = data['Close'] - data['SMA_10']
data['F_SMA_20'] = data['Close'] - data['SMA_20']

In [20]:
data = data.drop_duplicates()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [21]:
data.tail()

,Date,Time,f_time,julian_date,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close,SMA_5,SMA_10,SMA_20,F_SMA_5,F_SMA_10,F_SMA_20
9660,2022-03-11,02:00:00,2,70,4,Friday,2022-03-11T02:00:00.000000000Z,6495,0.93049,0.93096,0.92951,0.93068,0.9304,0.9287,0.9284,0.00028,0.00198,0.00228
9661,2022-03-11,06:00:00,6,70,4,Friday,2022-03-11T06:00:00.000000000Z,14714,0.93068,0.93209,0.92950,0.93184,0.9308,0.9293,0.9287,0.00104,0.00254,0.00314
9662,2022-03-11,10:00:00,10,70,4,Friday,2022-03-11T10:00:00.000000000Z,21746,0.93188,0.93204,0.92956,0.93144,0.9310,0.9298,0.9290,0.00044,0.00164,0.00244
9663,2022-03-11,14:00:00,14,70,4,Friday,2022-03-11T14:00:00.000000000Z,18826,0.93144,0.93473,0.93124,0.93408,0.9317,0.9304,0.9292,0.00238,0.00368,0.00488
9664,2022-03-11,18:00:00,18,70,4,Friday,2022-03-11T18:00:00.000000000Z,9554,0.93406,0.93500,0.93340,0.93484,0.9326,0.9311,0.9296,0.00224,0.00374,0.00524


## Price Range

In [22]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for value in data['col_1']:   
    if value > 0:
        data['col_2'] = data['High'] - data['Open']
        data['col_3'] = data['Close'] - data['Low']
    else:
        data['col_2'] = data['High'] - data['Close']
        data['col_3'] = data['Open'] - data['Low']

#Two Previous Candlesticks 
data['col_4'] = data['col_1'].shift(1)
data['col_5'] = data['col_1'].shift(2)

In [23]:
data = data.dropna()
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [ ]:
%%time
for candle_no in data.index.tolist():
#    print (candle_no)
    data.at[candle_no, "Trend"] = find_candle_trend(candle_no)

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data = data.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)